In [1]:
import pandas as pd
import re
from collections import Counter
from datetime import datetime
import calendar

In [2]:
def conv(s):
    s = s[1:-1]
    dic = {}

    positions=[match.end() for match in re.finditer(r'\b(?:positive|negative|neutral)\b,', s, flags=re.IGNORECASE)]
    for ss in [s[i:j] for i, j in zip([0] + positions, positions + [None])]:
        tmp = ss.split(':')
        try:
            dic[tmp[0]] = tmp[1].replace(",", "")
        except:
            continue

    return dic

def prepros(df):
    df.sentiments = df.sentiments.str.replace('[\n"\']', '', regex=True)
    df.sentiments = df.sentiments.str.replace(r'{\s+', '{', regex=True)
    df.sentiments = df.sentiments.str.replace(r'\s+}', '}', regex=True)
    df.sentiments = df.sentiments.str.replace(r':\s+', ':', regex=True)
    df.sentiments = df.sentiments.str.replace(r',\s+', ',', regex=True)
    df.sentiments = df.sentiments.apply(conv)

    return df

def map_to_root(df, mapping):
    top = list(mapping.index)
    top_dic = {}
    for dic in df.sentiments:
        for k in dic:
            #print(k)
            if k in top:
                #print(mapping.loc[k].loc['map'])
                try:
                    top_dic[mapping.loc[k].loc['map']] += [dic[k]]
                except:
                    #print(k)
                    #print(mapping.loc[k])
                    top_dic[mapping.loc[k].loc['map']] = [dic[k]]

    return top_dic

def sent_count(l):
    dic = {'positive': 0, 'negative': 0, 'neutral': 0}
    for i in l:
        try:
            dic[i] +=1
        except:
            continue

    return dic

def polarity_score(d):
    P = d['positive']
    N = d['negative']
    T = sum(d.values())

    try:
        return (P-N)/T #(P, N, T-P-N)
    except:
        return 0

In [3]:
def Main(f, y, m):
    mapping = pd.read_excel('Top Topics/'+f)
    mapping = mapping[mapping.include==1]

    mapping.index = list(mapping.ent)

    df = pd.read_excel('Topic Sentiment Data/'+f)

    if(y!='all'):
        df = df[df.date_year==y]
        #df = df[df.date_month.isin(m)]

    df = prepros(df)

    top_senti_list = map_to_root(df, mapping)

    for k in top_senti_list:
        top_senti_list[k] = sent_count(top_senti_list[k])

    top_senti_list = dict(sorted(top_senti_list.items(), key=lambda x: x[1]['positive'] + x[1]['negative'] + x[1]['neutral'], reverse=True))

    return top_senti_list

In [4]:
files = ['checkyourfact.xlsx',
 'politifact.xlsx',
 'snopes.xlsx',
 'altnews.xlsx',
 'boomlive.xlsx',
 'opindia.xlsx']

for f in files:
    top_senti_all = Main(f, 'all', 'all')

    alloverall = {'positive': 0, 'negative': 0, 'neutral': 0}

    for i in top_senti_all:
        tmp = top_senti_all[i]
        for j in tmp:
            try:
                alloverall[j] += tmp[j]
            except:
                print(i,j)

    print(f, "\t", polarity_score(alloverall))

checkyourfact.xlsx 	 -0.11610169491525424
politifact.xlsx 	 -0.10361752408652972
snopes.xlsx 	 -0.27574266447967927
altnews.xlsx 	 -0.2747336377473364
boomlive.xlsx 	 -0.18501805054151624
opindia.xlsx 	 -0.24912689173457508
